# Setup

In [1]:
!nvidia-smi

Sat Apr  3 08:31:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)
import os
root_path = 'gdrive/My Drive/HateXplain/'
os.chdir(root_path)

Mounted at /content/gdrive


In [3]:
# !pip install sentencepiece==0.1.94
# !pip install transformers
# !pip install ekphrasis
# !pip install datasets

In [4]:
import numpy as np
from datasets import list_datasets, load_dataset
import pandas as pd
import pickle
from transformers import AutoModel, AutoTokenizer
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.autograd import Function
import copy
from transformers import BertModel, RobertaModel, BertTokenizer, RobertaTokenizer, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, RandomSampler, SequentialSampler, random_split, DataLoader, IterableDataset, ConcatDataset
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import pickle
import json
from sklearn.metrics import accuracy_score
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import re
plt.rcParams['figure.figsize'] = [15, 8]
plt.rcParams.update({'font.size': 8})
RANDOM_SEED = 42
model_path = 'bert-base-uncased'
# model_path = 'monsoon-nlp/tamillion'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
random_seed(RANDOM_SEED, True)
%load_ext autoreload
%autoreload 2

In [6]:
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    # terms that will be annotated
    fix_html=True,  # fix HTML tokens
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    #corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading english - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/english/counts_1grams.txt


In [7]:
def split_stratified_into_train_val_test(df_input, stratify_colname='y',
                                         frac_train=0.6, frac_val=0.15, frac_test=0.25,
                                         random_state=None):
    if frac_train + frac_val + frac_test != 1.0:
        raise ValueError('fractions %f, %f, %f do not add up to 1.0' % \
                         (frac_train, frac_val, frac_test))

    if stratify_colname not in df_input.columns:
        raise ValueError('%s is not a column in the dataframe' % (stratify_colname))

    X = df_input # Contains all columns.
    y = df_input[[stratify_colname]] # Dataframe of just the column on which to stratify.

    # Split original dataframe into train and temp dataframes.
    df_train, df_temp, y_train, y_temp = train_test_split(X,
                                                          y,
                                                          stratify=y,
                                                          test_size=(1.0 - frac_train),
                                                          random_state=random_state)

    # Split the temp dataframe into val and test dataframes.
    relative_frac_test = frac_test / (frac_val + frac_test)
    df_val, df_test, y_val, y_test = train_test_split(df_temp,
                                                      y_temp,
                                                      stratify=y_temp,
                                                      test_size=relative_frac_test,
                                                      random_state=random_state)

    assert len(df_input) == len(df_train) + len(df_val) + len(df_test)

    return df_train, df_val, df_test

In [8]:
dataset = load_dataset('hatexplain', split = ['train', 'validation', 'test'])

train_dataset = dataset[0]
valid_dataset = dataset[1]
test_dataset = dataset[2]

Dataset hatexplain downloaded and prepared to /root/.cache/huggingface/datasets/hatexplain/plain_text/1.0.0/802fcd855438812094e336cea509c99b04b890e4e0846c0385877ee2c7361e93. Subsequent calls will reuse this data.


In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

In [10]:
import numpy as np
from numpy import array, exp
###this file contain different attention mask calculation from the n masks from n annotators. In this code there are 3 annotators

##### We mostly use softmax
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)
 
def neg_softmax(x):
    """Compute softmax values for each sets of scores in x. Here we convert the exponentials to 1/exponentials"""
    e_x = np.exp(-(x - np.max(x)))
    return e_x / e_x.sum(axis=0)
def sigmoid(z):
      """Compute sigmoid values"""
      g = 1 / (1 + exp(-z))
      return g

##### This function is used to aggregate the attentions vectors. This has a lot of options refer to the parameters explanation for understanding each parameter.
def aggregate_attention(at_mask,row):
    """input: attention vectors from 2/3 annotators (at_mask), row(dataframe row), params(parameters_dict)
       function: aggregate attention from different annotators.
       output: aggregated attention vector"""
    
    
    #### If the final label is normal or non-toxic then each value is represented by 1/len(sentences)
    if(row['final_label'] in ['normal','non-toxic']):
        at_mask_fin=[1/len(at_mask[0]) for x in at_mask[0]]
    else:
        at_mask_fin=at_mask
        #### Else it will choose one of the options, where variance is added, mean is calculated, finally the vector is normalised.   
        at_mask_fin=int(5)*at_mask_fin
        at_mask_fin=np.mean(at_mask_fin,axis=0)
        at_mask_fin=softmax(at_mask_fin)

    return at_mask_fin

In [11]:
def custom_tokenize(sent,tokenizer,max_length=512):
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    try:

        encoded_sent = tokenizer.encode(
                            sent,                      # Sentence to encode.
                            add_special_tokens = False, # Add '[CLS]' and '[SEP]'
                            #max_length = max_length,
                            # This function also supports truncation and conversion
                            # to pytorch tensors, but we need to do padding, so we
                            # can't use these features :( .
                            #max_length = 128,          # Truncate all sentences.
                            #return_tensors = 'pt',     # Return pytorch tensors.
                       )

        # Add the encoded sentence to the list.

    except ValueError:
        encoded_sent = tokenizer.encode(
                            ' ',                      # Sentence to encode.
                            add_special_tokens = False, # Add '[CLS]' and '[SEP]'
                            max_length = max_length,
                    
                       )
          ### decide what to later

    return encoded_sent

def ek_extra_preprocess(text,tokenizer):
    remove_words=['<allcaps>','</allcaps>','<hashtag>','</hashtag>','<elongated>','<emphasis>','<repeated>','\'','s']
    word_list=text_processor.pre_process_doc(text)
    # if(params['include_special']):
    #     pass
    # else:
    word_list=list(filter(lambda a: a not in remove_words, word_list)) 
    # if(params['bert_tokens']):
    sent=" ".join(word_list)
    sent = re.sub(r"[<\*>]", " ",sent)
    sub_word_list = custom_tokenize(sent,tokenizer)
    return sub_word_list

In [12]:
def returnMask(row,tokenizer):
    
    text_tokens=row['post_tokens']
    
    
    
    ##### a very rare corner case
    if(len(text_tokens)==0):
        text_tokens=['dummy']
        print("length of text ==0")
    #####
    
    
    mask_all= row['rationales']
    mask_all_temp=mask_all
    count_temp=0
    while(len(mask_all_temp)!=3):
        mask_all_temp.append([0]*len(text_tokens))
    
    word_mask_all=[]
    word_tokens_all=[]
    
    for mask in mask_all_temp:
        if(mask[0]==-1):
            mask=[0]*len(mask)
        
        
        list_pos=[]
        mask_pos=[]
        
        flag=0
        for i in range(0,len(mask)):
            if(i==0 and mask[i]==0):
                list_pos.append(0)
                mask_pos.append(0)
            
            
            
            
            if(flag==0 and mask[i]==1):
                mask_pos.append(1)
                list_pos.append(i)
                flag=1
                
            elif(flag==1 and mask[i]==0):
                flag=0
                mask_pos.append(0)
                list_pos.append(i)
        if(list_pos[-1]!=len(mask)):
            list_pos.append(len(mask))
            mask_pos.append(0)
        string_parts=[]
        for i in range(len(list_pos)-1):
            string_parts.append(text_tokens[list_pos[i]:list_pos[i+1]])
        
        
        word_tokens=[101]
        word_mask=[0]

        
        for i in range(0,len(string_parts)):
            tokens=ek_extra_preprocess(" ".join(string_parts[i]),tokenizer)
            masks=[mask_pos[i]]*len(tokens)
            word_tokens+=tokens
            word_mask+=masks


        # if(params['bert_tokens']):
        ### always post truncation
        word_tokens=word_tokens[0:(int(128)-2)]
        word_mask=word_mask[0:(int(128)-2)]
        word_tokens.append(102)
        word_mask.append(0)

        word_mask_all.append(word_mask)
        word_tokens_all.append(word_tokens)
        
#     for k in range(0,len(mask_all)):
#          if(mask_all[k][0]==-1):
#             word_mask_all[k] = [-1]*len(word_mask_all[k])
    if(len(mask_all)==0):
        word_mask_all=[]
    else:    
        word_mask_all=word_mask_all[0:len(mask_all)]
    return word_tokens_all[0],word_mask_all

In [13]:
dict_label = {0: 'hatespeech', 1: 'normal', 2:'offensive'}

In [14]:
class Dataset():
    def __init__(self, data, batch_size = 16, train = False):
        self.data = data
        # self.val_data = val_data
        self.batch_size = batch_size
        self.train = train
        # self.label_dict = {0: 0,
        #                     1: 2,
        #                     2: 1}
                                    
        self.count_dic = {}
        self.inputs, self.labels, self.attn = self.process_data(self.data)
        self.DataLoader = self.get_dataloader(self.inputs, self.attn, self.labels)
        # self.train_dataloader = self.process_data(dataset_file, post_id_divisions_file, 'train')
        # self.val_dataloader = self.process_data(dataset_file, post_id_divisions_file, 'test')
        # self.test_dataloader = self.process_data(dataset_file, post_id_divisions_file, 'test')

    def tokenize(self, sentences, padding = True, max_len = 128):
        tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
        input_ids, attention_masks, token_type_ids = [], [], []
        for sent in sentences:
            encoded_dict = tokenizer.encode_plus(sent,
                                                    add_special_tokens=True,
                                                    max_length=max_len, 
                                                    padding='max_length', 
                                                    return_attention_mask = True,
                                                    return_tensors = 'pt', 
                                                    truncation = True)
            input_ids.append(encoded_dict['input_ids'])
            attention_masks.append(encoded_dict['attention_mask'])
        
        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)

        return {'input_ids': input_ids, 'attention_masks': attention_masks}
    
    def process_data(self, data):
        sentences, labels, attn = [], [], []
        print(len(data))
        for row in data:
            word_tokens_all, word_mask_all = returnMask(row, tokenizer)
            label = max(set(row['annotators']['label']), key = row['annotators']['label'].count)
            # label = self.label_dict[label]
            sentence = ' '.join(row['post_tokens'])
            sentences.append(sentence)
            labels.append(label)
            row['final_label'] = dict_label[label]
            at_mask = np.array(aggregate_attention(word_mask_all,row))
            at_mask = at_mask.tolist() + [0]*(128-len(at_mask))

            attn.append(at_mask)
        inputs = self.tokenize(sentences)
        return inputs, torch.Tensor(labels), torch.Tensor(attn)
    
    def get_dataloader(self, inputs, attn, labels, train = True):
        data = TensorDataset(inputs['input_ids'], inputs['attention_masks'], attn, labels)
        if self.train:
            sampler = RandomSampler(data)
        else:
            sampler = SequentialSampler(data)
        return DataLoader(data, sampler=sampler, batch_size=self.batch_size, drop_last=True)

In [15]:
train_data_source = Dataset(train_dataset, train = True)
val_data_source = Dataset(valid_dataset)
test_data_source = Dataset(test_dataset)

15383
1922
1924


In [38]:
Counter(train_data_source.labels.tolist())Counter({0.0: 4748, 1.0: 6251, 2.0: 4384})

Counter({0.0: 4748, 1.0: 6251, 2.0: 4384})

In [16]:
class Dataset():
    def __init__(self, data, batch_size = 16, train = False, max_len = 128):
        self.data = data
        # self.val_data = val_data
        self.batch_size = batch_size
        self.train = train
        self.label_dict = {0: 0,
                            1: 2,
                            2: 1}
                                    
        self.count_dic = {}
        self.inputs, self.labels = self.process_data(self.data)
        self.DataLoader = self.get_dataloader(self.inputs, self.labels)
        # self.train_dataloader = self.process_data(dataset_file, post_id_divisions_file, 'train')
        # self.val_dataloader = self.process_data(dataset_file, post_id_divisions_file, 'test')
        # self.test_dataloader = self.process_data(dataset_file, post_id_divisions_file, 'test')

    def ek_extra_preprocess(self, text):
        remove_words=['<allcaps>','</allcaps>','<hashtag>','</hashtag>','<elongated>','<emphasis>','<repeated>','\'','s']
        word_list=text_processor.pre_process_doc(text)
        word_list=list(filter(lambda a: a not in remove_words, word_list)) 
        sent=" ".join(word_list)
        sent = re.sub(r"[<\*>]", " ",sent)
        return sent

    def tokenize(self, sentences, padding = True, max_len = 128):
        tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
        input_ids, attention_masks, token_type_ids = [], [], []
        for sent in sentences:
            encoded_dict = tokenizer.encode_plus(sent,
                                                    add_special_tokens=True,
                                                    max_length=max_len, 
                                                    padding='max_length', 
                                                    return_attention_mask = True,
                                                    return_tensors = 'pt', 
                                                    truncation = True)
            input_ids.append(encoded_dict['input_ids'])
            attention_masks.append(encoded_dict['attention_mask'])
        
        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)

        return {'input_ids': input_ids, 'attention_masks': attention_masks}
    
    def process_data(self, data):
        sentences, labels = [], []
        print(len(data))
        for label, sentence in zip(list(data['class']), list(data['tweet'])):
            label = self.label_dict[label]
            sentence = self.ek_extra_preprocess(sentence)
            sentences.append(sentence)
            labels.append(label)
        inputs = self.tokenize(sentences)
        return inputs, torch.Tensor(labels)
    
    def get_dataloader(self, inputs, labels, train = True):
        data = TensorDataset(inputs['input_ids'], inputs['attention_masks'], labels)
        if self.train:
            sampler = RandomSampler(data)
        else:
            sampler = SequentialSampler(data)
        return DataLoader(data, sampler=sampler, batch_size=self.batch_size, drop_last=True)

In [17]:
# df = pd.read_csv('Data/labeled_data.csv', sep=',', engine='python')
# df = df.rename(columns={'Unnamed: 0': 'id'})
# df_train, df_val, df_test = split_stratified_into_train_val_test(df, stratify_colname='class',
#                                          frac_train=0.8, frac_val=0.1, frac_test=0.1,
#                                          random_state=RANDOM_SEED)
# df_train.to_csv('Data/train_data.csv',index = False)
# df_val.to_csv('Data/val_data.csv',index = False)
# df_test.to_csv('Data/test_data.csv',index = False)

In [28]:
df_train = pd.read_csv('Data/train_data.csv', sep=',', engine='python')
df_val = pd.read_csv('Data/val_data.csv', sep=',', engine='python')
df_test = pd.read_csv('Data/test_data.csv', sep=',', engine='python')

In [33]:
from collections import Counter
Counter(df_train['class'])

Counter({0: 1144, 1: 15352, 2: 3330})

In [19]:
class GradReverse(Function):
    @staticmethod
    def forward(ctx, x, lambd):
        ctx.lambd = lambd
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        return (grad_output * -ctx.lambd), None


def grad_reverse(x, lambd=1.0):
    return GradReverse.apply(x, lambd)


def calc_coeff(iter_num, high=1.0, low=0.0, alpha=10.0, max_iter=10000.0):
    return np.float(2.0 * (high - low) /
                    (1.0 + np.exp(-alpha * iter_num / max_iter)) -
                    (high - low) + low)


def entropy(F1, feat, lamda, eta=1.0):
    out_t1 = F1(feat, reverse=True, eta=-eta)
    out_t1 = F.softmax(out_t1)
    loss_ent = -lamda * torch.mean(torch.sum(out_t1 *
                                             (torch.log(out_t1 + 1e-5)), 1))
    return loss_ent


def adentropy(F1, feat, lamda, eta=1.0):
    out_t1 = F1(feat, reverse=True, eta=eta)
    out_t1 = F.softmax(out_t1)
    loss_adent = lamda * torch.mean(torch.sum(out_t1 *
                                              (torch.log(out_t1 + 1e-5)), 1))
    return loss_adent

class Predictor(nn.Module):
    def __init__(self, num_class=64, inc=4096, temp=0.05):
        super(Predictor, self).__init__()
        self.fc = nn.Linear(inc, num_class, bias=False)
        self.num_class = num_class
        self.temp = temp

    def forward(self, x, reverse=False, eta=0.1):
        if reverse:
            x = grad_reverse(x, eta)
        x = F.normalize(x)
        x_out = self.fc(x) / self.temp
        return x_out

In [20]:
def cross_entropy(input1, target, size_average=True):
    logsoftmax = nn.LogSoftmax(dim=0)
    return torch.sum(-target * logsoftmax(input1))

def masked_cross_entropy(input1,target,mask):
    cr_ent=0
    for h in range(0,mask.shape[0]):
        cr_ent+=cross_entropy(input1[h][mask[h]],target[h][mask[h]])
    
    return cr_ent/mask.shape[0]

class SC_weighted_BERT(nn.Module):
    def __init__(self, model_path):
        super(SC_weighted_BERT, self).__init__()
        self.num_labels = 3
        self.weights=[1.0795518,  0.82139814, 1.1678787]
        self.train_att= True
        self.lam = 100
        self.num_sv_heads = 6
        self.sv_layer = 11
        self.bert = AutoModel.from_pretrained(model_path, output_attentions = True)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768, 3)
        #self.softmax=nn.Softmax(config.num_labels)
        # self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, attn = None, attn_lam = 0):

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
        )

        pooled_output = outputs[1]
        if attn is None:
            return pooled_output

        outputs = (pooled_output,) + outputs[2:]  # add hidden states and attention if they are here
        loss= 0
        if(self.train_att):
            loss_att=0
            for i in range(self.num_sv_heads):
                attention_weights=outputs[1][self.sv_layer][:,i,0,:]
                loss_att +=attn_lam*masked_cross_entropy(attention_weights,attn,attention_mask)
            loss = loss + loss_att
        outputs = (loss,) + outputs

        return outputs  # (loss), (pooled output), etc.

In [21]:
def train():
    G.train()
    F1.train()
    optimizer_g = optim.SGD(params, momentum=0.9,
                            weight_decay=0.0005, nesterov=True)
    optimizer_f = optim.SGD(list(F1.parameters()), lr=1.0, momentum=0.9,
                            weight_decay=0.0005, nesterov=True)

    def zero_grad_all():
        optimizer_g.zero_grad()
        optimizer_f.zero_grad()
    param_lr_g = []
    for param_group in optimizer_g.param_groups:
        param_lr_g.append(param_group["lr"])
    param_lr_f = []
    for param_group in optimizer_f.param_groups:
        param_lr_f.append(param_group["lr"])
    criterion = nn.CrossEntropyLoss().cuda()
    all_step = args.steps
    data_iter_s = iter(source_loader)
    data_iter_t = iter(target_loader)
    data_iter_t_unl = iter(target_loader_unl)
    len_train_source = len(source_loader)
    len_train_target = len(target_loader)
    len_train_target_semi = len(target_loader_unl)
    best_acc = 0
    counter = 0
    for step in range(all_step):
        optimizer_g = inv_lr_scheduler(param_lr_g, optimizer_g, step,
                                       init_lr=args.lr)
        optimizer_f = inv_lr_scheduler(param_lr_f, optimizer_f, step,
                                       init_lr=args.lr)
        lr = optimizer_f.param_groups[0]['lr']
        if step % len_train_target == 0:
            data_iter_t = iter(target_loader)
        if step % len_train_target_semi == 0:
            data_iter_t_unl = iter(target_loader_unl)
        if step % len_train_source == 0:
            data_iter_s = iter(source_loader)
        data_t = next(data_iter_t)
        data_t_unl = next(data_iter_t_unl)
        data_s = next(data_iter_s)
        im_data_s.data.resize_(data_s[0].size()).copy_(data_s[0])
        gt_labels_s.data.resize_(data_s[1].size()).copy_(data_s[1])
        im_data_t.data.resize_(data_t[0].size()).copy_(data_t[0])
        gt_labels_t.data.resize_(data_t[1].size()).copy_(data_t[1])
        im_data_tu.data.resize_(data_t_unl[0].size()).copy_(data_t_unl[0])
        zero_grad_all()
        data = torch.cat((im_data_s, im_data_t), 0)
        target = torch.cat((gt_labels_s, gt_labels_t), 0)
        output = G(data)
        out1 = F1(output)
        loss = criterion(out1, target)
        loss.backward(retain_graph=True)
        optimizer_g.step()
        optimizer_f.step()
        zero_grad_all()
        if not args.method == 'S+T':
            output = G(im_data_tu)
            if args.method == 'ENT':
                loss_t = entropy(F1, output, args.lamda)
                loss_t.backward()
                optimizer_f.step()
                optimizer_g.step()
            elif args.method == 'MME':
                loss_t = adentropy(F1, output, args.lamda)
                loss_t.backward()
                optimizer_f.step()
                optimizer_g.step()
            else:
                raise ValueError('Method cannot be recognized.')
            log_train = 'S {} T {} Train Ep: {} lr{} \t ' \
                        'Loss Classification: {:.6f} Loss T {:.6f} ' \
                        'Method {}\n'.format(args.source, args.target,
                                             step, lr, loss.data,
                                             -loss_t.data, args.method)
        else:
            log_train = 'S {} T {} Train Ep: {} lr{} \t ' \
                        'Loss Classification: {:.6f} Method {}\n'.\
                format(args.source, args.target,
                       step, lr, loss.data,
                       args.method)
        G.zero_grad()
        F1.zero_grad()
        zero_grad_all()
        if step % args.log_interval == 0:
            print(log_train)
        if step % args.save_interval == 0 and step > 0:
            loss_test, acc_test = test(target_loader_test)
            loss_val, acc_val = test(target_loader_val)
            G.train()
            F1.train()
            if acc_val >= best_acc:
                best_acc = acc_val
                best_acc_test = acc_test
                counter = 0
            else:
                counter += 1
            if args.early:
                if counter > args.patience:
                    break
            print('best acc test %f best acc val %f' % (best_acc_test,
                                                        acc_val))
            print('record %s' % record_file)
            with open(record_file, 'a') as f:
                f.write('step %d best %f final %f \n' % (step,
                                                         best_acc_test,
                                                         acc_val))
            G.train()
            F1.train()
            if args.save_check:
                print('saving model')
                torch.save(G.state_dict(),
                           os.path.join(args.checkpath,
                                        "G_iter_model_{}_{}_"
                                        "to_{}_step_{}.pth.tar".
                                        format(args.method, args.source,
                                               args.target, step)))
                torch.save(F1.state_dict(),
                           os.path.join(args.checkpath,
                                        "F1_iter_model_{}_{}_"
                                        "to_{}_step_{}.pth.tar".
                                        format(args.method, args.source,
                                               args.target, step)))

In [22]:
import copy
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
 
def get_predicted(preds):
    pred_flat = np.argmax(preds, axis=1).flatten()
    return pred_flat
 
def evaluate(test_dataloader, feature_encoder, classifer):
    feature_encoder.eval()
    classifier.eval()

    y_preds, y_test = np.array([]), np.array([])

    for batch in test_dataloader:
        b_input_ids, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device).long()
        with torch.no_grad():        
            ypred = feature_encoder(b_input_ids, b_input_mask)
            ypred = classifier(ypred)

        ypred = ypred.cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        y_preds = np.hstack((y_preds, get_predicted(ypred)))
        y_test = np.hstack((y_test, label_ids))

    weighted_f1 = f1_score(y_test, y_preds, average='macro')
    report = classification_report(y_test, y_preds)
    print(report)
    return weighted_f1, y_preds, y_test
 
def train(source_dataloader, target_labelled_dataloader, target_unlabelled_dataloader, validation_dataloader, 
          feature_encoder, classifier, attn_lam = 0.01, learning_rate = 2e-5, steps = 1000):
    losses = []
    total_steps = steps
    no_decay = ['bias', 'LayerNorm.weight', 'LayerNorm.bias']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in feature_encoder.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in feature_encoder.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

    optimizer_feature_encoder = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps = 1e-8)
    scheduler_feature_encoder = get_linear_schedule_with_warmup(optimizer_feature_encoder, 
                                                num_warmup_steps = 0, # Default value in run_glue.py
                                                num_training_steps = total_steps)
    
    optimizer_classifier = AdamW(list(classifier.parameters()), lr=learning_rate*10, eps = 1e-8)
    scheduler_classifier = get_linear_schedule_with_warmup(optimizer_classifier, 
                                                num_warmup_steps = 0, # Default value in run_glue.py
                                                num_training_steps = total_steps)
    
    
    best_weighted_f1 = 0
    best_feature_encoder, best_classifier = None, None
    # current_epoch, best_weighted_f1 = load_metrics(filepath, model, optimizer)
    criterion = nn.CrossEntropyLoss()

    len_source = len(source_dataloader)
    len_target_l = len(target_labelled_dataloader)
    len_target_ul = len(target_unlabelled_dataloader)

    feature_encoder.train()
    classifier.train()

    for step in range(total_steps):
        if step % len_source == 0:
            data_iter_s = iter(source_dataloader)
        if step % len_target_l == 0:
            data_iter_tl = iter(target_labelled_dataloader)
        if step % len_target_ul == 0:
            data_iter_tu = iter(target_unlabelled_dataloader)

        batch_s = next(data_iter_s)
        batch_tl = next(data_iter_tl)
        batch_tu = next(data_iter_tu)

        b_input_ids_s, b_input_mask_s, b_attn_s, b_labels_s = batch_s[0].to(device), batch_s[1].to(device), batch_s[2].to(device), batch_s[3].to(device).long()
        b_input_ids_tl, b_input_mask_tl, b_labels_tl = batch_tl[0].to(device), batch_tl[1].to(device), batch_tl[2].to(device).long()
        b_input_ids_tu, b_input_mask_tu, b_labels_tu = batch_tu[0].to(device), batch_tu[1].to(device), batch_tu[2].to(device).long()
        
        out1 = feature_encoder(b_input_ids_s, b_input_mask_s, attn = b_attn_s, attn_lam = attn_lam)
        logits1 = classifier(out1[1])

        out2 = feature_encoder(b_input_ids_tl, b_input_mask_tl)
        logits2 = classifier(out2)
        
        loss_attn = out1[0]

        loss = criterion(torch.cat([logits1, logits2], dim = 0), torch.cat([b_labels_s, b_labels_tl], dim = 0)) + loss_attn

        output = feature_encoder(b_input_ids_tu, b_input_mask_tu)
        loss_t = adentropy(classifier, output, 0.1)
        loss += loss_t
        
        if step%50 == 0:
            print(loss.item())
            losses.append(float(loss))
            print('### Validation Set Stats')
            weighted_f1, ypred, ytest = evaluate(validation_dataloader, feature_encoder, classifier)
            print("  Macro F1: {0:.3f}".format(weighted_f1))
            if weighted_f1 > best_weighted_f1:
                best_weighted_f1 = weighted_f1
                best_feature_encoder = copy.deepcopy(feature_encoder)
                best_classifier = copy.deepcopy(classifier)
                # save_metrics(filepath, epoch_i, model, optimizer, weighted_f1)

        optimizer_feature_encoder.zero_grad()
        optimizer_classifier.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(feature_encoder.parameters(), 1.0)
        optimizer_feature_encoder.step()
        optimizer_classifier.step()

        scheduler_feature_encoder.step()
        scheduler_classifier.step() 
        
    return best_feature_encoder, best_classifier, losses

In [23]:
feature_encoder = SC_weighted_BERT(model_path).to(device)
classifier = Predictor(num_class=3, inc = 768).to(device)
# model.load_state_dict(torch.load('./Saved/bert-base-uncased_11_6_3_100/pytorch_model.bin', 'cpu'))

In [24]:
n = 50
train_data_target_unlabelled = Dataset(df_train, train = True)
val_data, test_data = Dataset(df_val), Dataset(df_test)

19826
2478
2479


In [25]:
# feature_encoder, classifier, losses = train(train_data_source.DataLoader, train_data_target_labelled.DataLoader,
#       train_data_target_unlabelled.DataLoader, val_data.DataLoader, 
#       feature_encoder, classifier, attn_lam = 0.01, learning_rate = 2e-5, steps = 5000)

In [26]:
try:
    with open('SSDA/results_davidson.pkl', 'rb') as f:
        data_dict = pickle.load(f)
except:
    data_dict = {}
    with open('SSDA/results_davidson.pkl', 'wb') as f:
        pickle.dump(data_dict, f)

In [27]:
attentions = [0, 0.001, 0.01, 0.1, 1, 10, 100]
N = [20, 50, 100, 200, 300, 400, 500, 1000]
for n in N:
    if n!=0:
        df = pd.concat([df_train[df_train['class']==0].sample(n=n, random_state = RANDOM_SEED),
                    df_train[df_train['class']==1].sample(n=n, random_state = RANDOM_SEED),
                    df_train[df_train['class']==2].sample(n=n, random_state = RANDOM_SEED)],
                    ignore_index = True)
        train_data_target_labelled = Dataset(df, batch_size = 16, train = True)

    for attn in attentions:
        if attn not in data_dict: data_dict[attn] = {}
        if n in data_dict[attn]: continue
        feature_encoder = SC_weighted_BERT(model_path).to(device)
        classifier = Predictor(num_class=3, inc = 768).to(device)
        if n!=0:
            feature_encoder, classifier, losses = train(train_data_source.DataLoader, train_data_target_labelled.DataLoader,
            train_data_target_unlabelled.DataLoader, val_data.DataLoader, 
            feature_encoder, classifier, attn_lam = 0.01, learning_rate = 2e-5, steps = 1000)

        f1, ypreds, ytest = evaluate(test_data.DataLoader, feature_encoder, classifier)
        acc = accuracy_score(ytest, ypreds)
        with open('SSDA/results_davidson.pkl', 'rb') as f:
            data_dict = pickle.load(f)
        if attn not in data_dict: data_dict[attn] = {}
        data_dict[attn][n] = {'f1': f1, 'acc':acc}
        print(attn, n, data_dict[attn][n])
        with open('SSDA/results_davidson.pkl', 'wb') as f:
            pickle.dump(data_dict, f)
        print(data_dict)

60
150
300
600
900
1200


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


2.7982540130615234
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       143
         1.0       0.17      0.01      0.01       414
         2.0       0.77      0.99      0.87      1907

    accuracy                           0.77      2464
   macro avg       0.31      0.33      0.29      2464
weighted avg       0.63      0.77      0.67      2464

  Macro F1: 0.294
2.4281558990478516
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.21      0.59      0.31       143
         1.0       0.58      0.67      0.62       414
         2.0       0.91      0.76      0.83      1907

    accuracy                           0.73      2464
   macro avg       0.57      0.67      0.59      2464
weighted avg       0.82      0.73      0.76      2464

  Macro F1: 0.587
2.390320062637329
### Validation Set Stats
              precision    recall  f1-score   support

         0.0    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


3.0729732513427734
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       143
         1.0       0.09      0.02      0.03       414
         2.0       0.77      0.96      0.85      1907

    accuracy                           0.74      2464
   macro avg       0.29      0.33      0.30      2464
weighted avg       0.61      0.74      0.67      2464

  Macro F1: 0.295
2.5160515308380127
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.21      0.48      0.30       143
         1.0       0.49      0.75      0.59       414
         2.0       0.91      0.72      0.80      1907

    accuracy                           0.71      2464
   macro avg       0.54      0.65      0.56      2464
weighted avg       0.80      0.71      0.74      2464

  Macro F1: 0.564
1.8197236061096191
### Validation Set Stats
              precision    recall  f1-score   support

         0.0   

...

              precision    recall  f1-score   support

         0.0       0.06      0.81      0.11       143
         1.0       0.12      0.16      0.14       414
         2.0       0.00      0.00      0.00      1907

    accuracy                           0.07      2464
   macro avg       0.06      0.32      0.08      2464
weighted avg       0.02      0.07      0.03      2464

  Macro F1: 0.083
2.4920740127563477
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.27      0.58      0.37       143
         1.0       0.54      0.75      0.62       414
         2.0       0.91      0.76      0.83      1907

    accuracy                           0.75      2464
   macro avg       0.57      0.70      0.61      2464
weighted avg       0.81      0.75      0.77      2464

  Macro F1: 0.608
2.2841482162475586
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.21      0.77      0.34       143
     

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


3.1720821857452393
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.07      0.05      0.06       143
         1.0       0.17      0.97      0.29       414
         2.0       0.80      0.00      0.01      1907

    accuracy                           0.17      2464
   macro avg       0.35      0.34      0.12      2464
weighted avg       0.65      0.17      0.06      2464

  Macro F1: 0.118
2.063913345336914
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.19      0.63      0.29       143
         1.0       0.46      0.70      0.55       414
         2.0       0.91      0.65      0.76      1907

    accuracy                           0.66      2464
   macro avg       0.52      0.66      0.54      2464
weighted avg       0.79      0.66      0.70      2464

  Macro F1: 0.535
1.871433973312378
### Validation Set Stats
              precision    recall  f1-score   support

         0.0     

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.06      0.98      0.11       143
         1.0       0.00      0.00      0.00       414
         2.0       0.81      0.02      0.04      1907

    accuracy                           0.07      2464
   macro avg       0.29      0.33      0.05      2464
weighted avg       0.63      0.07      0.03      2464

  Macro F1: 0.048
2.4699277877807617
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.21      0.25      0.23       143
         1.0       0.60      0.79      0.68       414
         2.0       0.92      0.84      0.88      1907

    accuracy                           0.80      2464
   macro avg       0.58      0.63      0.60      2464
weighted avg       0.82      0.80      0.81      2464

  Macro F1: 0.597
2.335071325302124
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.29      0.55      0.38       143
      